In [1]:
#installing external python packages 
!pip install pytesseract
!pip install opencv-python
!pip install watchdog


In [2]:
#importing libraries
import numpy as np 
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import pytesseract
import re 
import os
import time 
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
import shutil

In [3]:
def crop_img(img):
    """
        Crop and the image to the rectangle of the screen, and
    resize the image to a proper size.
    Args:
        img: image object
    Returns:
        cropped img
    """
    
    # Setting a threshold to find the screen (brightest part of image)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    screen = cv2.threshold(gray, 110, 255, cv2.THRESH_BINARY)[1]
    cv2.imshow('screen', screen)
    
    # finding the edges of the screen
    contours, _ = cv2.findContours(screen, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Selecting the largest rectangle
    areas = [cv2.contourArea(c) for c in contours]
    max_index = np.argmax(areas)

    #cropping the image
    cnt = contours[max_index]
    x,y,w,h = cv2.boundingRect(cnt)
    cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 2)
    cv2.imshow('crop img', img)
    img = img[y:y+h, x:x+w]

    return img


def isolate_red_text(img):
    """
        Isolating the red texts.
    Args:
        img: image object
    Returns:
        img only with red text
    """
    
    #creating first mask
    hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    cv2.imshow('hsv_img', hsv_img)
    lower = np.array([155,100,100])
    upper = np.array([175,255,255])
    mask = cv2.inRange(hsv_img, lower, upper)

    #creating second mask
    lower = np.array([0,100,100])
    upper = np.array([12,255,255])
    mask2 = cv2.inRange(hsv_img, lower, upper)
    mask |= mask2

    #using the mask to create image with only red text
    result = cv2.bitwise_and(img, img, mask=mask)
    return result


def identify_text(img):
    """
        Identify the text on image.
    Args:
        img: image object
    Returns:
        text: str
    """
    # Storing the processed image.
    cv2.imshow('identify text', img)
    cv2.waitKey(1000)
    outpath = "true.jpg"
    cv2.imwrite(outpath, img)
    cv2.destroyAllWindows()

    # Using tesseract to identify text.
    pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
    text = pytesseract.image_to_string(img)
    
    # Removing invalid charecters.
    text = ''.join([c for c in text if c not in "“‘—%|[]=_@_{}~€&£><;!?"])
    return text


def process_img(filename: str):
    
    #reading image
    img = cv2.imread(filename)
    image = cv2.imread(filename)
    image = crop_img(image)
    image = isolate_red_text(image)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # defining threshold to obtain binary image.
    _, image = cv2.threshold(gray, 30, 255, cv2.THRESH_BINARY_INV)

    return identify_text(image)   

In [ ]:
class ExampleHandler(FileSystemEventHandler):
    def on_created(self, event): # event created
        
        #defining source that recieve raspberry pi images
        path = r'C:\Users\dmjin\biodesignnew'
        path_file=os.listdir(path)
        old_checker=[]
        d=0
        
        #checking all image files in the source
        for j in range(len(path_file)):
            old_array=re.findall(r'[0-9]+', path_file[j])
            
            #checking how many image files
            if len(old_array)>0:
                new=old_array[0]
                old_checker.append(new)
                
                #identifying the image file
                if len(old_checker)>0:
                    max_loc=old_checker[0]
                    d+=1

                    #sending each image retreived to destination directory
                    if d==1:
                        joint_source=os.path.join(path, str(max_loc) + '.jpg')
                        destination = r'C:\Users\dmjin\working code'
                        joint_dest=os.path.join(destination, str(max_loc) + '.jpg')
                        dest = shutil.move(joint_source, joint_dest)
        
        
        #defining destination directory
        z=os.listdir('C:\\Users\\dmjin\\working code') 
        number_checker=[]
        e=0
        
        #finding and checking images retreived from raspberry pi
        for i in range(len(z)):
            array=re.findall(r'[0-9]+', z[i])
    
            #checking each file
            if len(array)>0:
                new=array[0]
                number_checker.append(new)
        
                #identifying the image file
                if len(number_checker)>0:
                    max_place=number_checker[0]
                    file_size = os.path.getsize(str(max_place) + '.jpg')
                    e+=1
                    
                    #processing image file if non-empty
                    if file_size!=0 and e==1:
                        text = process_img(str(max_place) + '.jpg')

                        #printing errors depending on text identified from image
                        if "AIR" in text:
                            print ("AIR-IN-LINE!")
                            os.remove(str(max_place) + '.jpg')
                        elif "DOWN" in text:
                            print ("DOWNSTREAM OCCLUSION!")
                            os.remove(str(max_place) + '.jpg')
                        elif "SLIDE" in text:
                            print ("SLIDE-IN-CLAMP!")
                            os.remove(str(max_place) + '.jpg')
                        elif "UP" in text:
                            print ("UPSTREAM OCCLUSION!")
                            os.remove(str(max_place) + '.jpg')
                        else:
                            print ("STATUS: NORMAL")
                            os.remove(str(max_place) + '.jpg')

                    #removing empty images
                    else:
                        os.remove(str(max_place) + '.jpg')

#monitoring directory and creating an event 
observer = Observer() 
event_handler = ExampleHandler() 

# setting the observer to use created handler in directory
observer.schedule(event_handler, path='C:\\Users\\dmjin\\biodesignnew',recursive=False)
observer.start()

#resting keyboard until interrupted, then stops and rejoins the observer if event occurs
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    observer.stop()

observer.join()

STATUS: NORMAL
STATUS: NORMAL
STATUS: NORMAL
AIR-IN-LINE!
